### Importar Librerias

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

### Cargar los datos

In [60]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

### Mostrar las columnas disponibles antes de seleccionar las características

In [61]:
print("Columnas disponibles en el dataset de entrenamiento:")
print(train_df.columns)

Columnas disponibles en el dataset de entrenamiento:
Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_in_euros'],
      dtype='object')


### Seleccionar columnas relevantes

In [62]:
features = ["Inches", "Ram", "Weight", "Gpu", "Cpu"]
target = "Price_in_euros"

- "Inches": El tamaño de la pantalla puede influir en el precio, ya que los modelos más grandes suelen ser más costosos.
- "Ram": La cantidad de memoria RAM es un factor clave en el rendimiento de una laptop y su precio.
- "Weight": El peso puede estar relacionado con la calidad de los materiales y la portabilidad, lo que puede impactar el precio.
- "Gpu": La tarjeta gráfica es un componente crucial, especialmente para laptops de alto rendimiento o gaming, y afecta directamente el costo.
- "Cpu": El procesador es un factor determinante en el rendimiento y también en el precio.


El target es "Price_in_euros" porque es la variable que queremos predecir.

### Convertir 'Ram' y 'Weight' a valores numéricos

In [63]:
train_df["Ram"] = train_df["Ram"].astype(str).str.replace("GB", "").astype(int)
train_df["Weight"] = train_df["Weight"].astype(str).str.replace("kg", "").astype(float)
test_df["Ram"] = test_df["Ram"].astype(str).str.replace("GB", "").astype(int)
test_df["Weight"] = test_df["Weight"].astype(str).str.replace("kg", "").astype(float)

### Manejo de variables categóricas con One-Hot Encoding

In [64]:
categorical_features = ["Gpu", "Cpu"]
numeric_features = ["Inches", "Ram", "Weight"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

### Codificar variables categóricas con Target Encoding

In [65]:
# def target_encode(df, col, target):
#     encoding = df.groupby(col)[target].mean()
#     return df[col].map(encoding)

# train_df["Gpu"] = target_encode(train_df, "Gpu", target)
# test_df["Gpu"] = test_df["Gpu"].map(train_df.groupby("Gpu")[target].mean()).fillna(train_df[target].mean())
# train_df["Cpu"] = target_encode(train_df, "Cpu", target)
# test_df["Cpu"] = test_df["Cpu"].map(train_df.groupby("Cpu")[target].mean()).fillna(train_df[target].mean())

### Limitar los precios desde el inicio

In [66]:
# train_df[target] = np.clip(train_df[target], None, 400)

### Codificar variables categóricas

In [67]:
# train_df = pd.get_dummies(train_df, columns=["Gpu", "Cpu"], drop_first=True)
# test_df = pd.get_dummies(test_df, columns=["Gpu", "Cpu"], drop_first=True)

### Alinear columnas de test y train

In [68]:
# test_df = test_df.reindex(columns=train_df.columns.drop(target), fill_value=0)

### Dividir los datos en entrenamiento y validación

In [69]:
X = train_df[features]
y = train_df[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Escalar los datos con RobustScaler para mayor resistencia a valores atípicos

In [70]:
# scaler = RobustScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(test_df.drop(columns=["laptop_ID"]))


### Construcción del modelo con Pipeline

In [71]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", GradientBoostingRegressor(
        n_estimators=1500, max_depth=8, learning_rate=0.01, subsample=0.8,
        min_samples_split=4, min_samples_leaf=1, max_features="sqrt",
        loss="huber", random_state=42))
])

### Entrenar el modelo

In [72]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['Inches', 'Ram', 'Weight']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gpu', 'Cpu'])])),
                ('regressor',
                 GradientBoostingRegressor(learning_rate=0.01, loss='huber',
                                           max_depth=8, max_features='sqrt',
                                           min_samples_split=4,
                                           n_estimators=1500, random_state=42,
                                           subsample=0.8))])

### Evaluar el modelo

In [73]:
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE en validación: {rmse}")

RMSE en validación: 387.19761449506666


### Predecir en el conjunto de prueba

In [74]:
X_test = test_df[features]
y_test_pred = model.predict(X_test)

### Crear archivo de predicción

In [79]:
submission = pd.DataFrame({"laptop_ID": test_df["laptop_ID"], "Price_in_euros": y_test_pred})
submission.to_csv("submission.csv", index=False)

In [80]:
print("Archivo submission.csv generado correctamente.")

Archivo submission.csv generado correctamente.


### Seleccionar solo columnas numéricas

In [ ]:
# numeric_features = X_train.select_dtypes(include=[np.number]).columns

# X_train_scaled = scaler.fit_transform(X_train[numeric_features])
# X_val_scaled = scaler.transform(X_val[numeric_features])
# X_test_scaled = scaler.transform(test_df[numeric_features])

### Entrenar el modelo con Gradient Boosting

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# model = GradientBoostingRegressor(n_estimators=200, max_depth=7, learning_rate=0.1, random_state=42)
# model.fit(X_train_scaled, y_train)

### Evaluar el modelo

In [ ]:
# y_pred = model.predict(X_val_scaled)
# rmse = np.sqrt(mean_squared_error(y_val, y_pred))
# print(f"RMSE en validación: {rmse}")

RMSE en validación: 29.547195486734324


### Predecir en el conjunto de prueba

In [ ]:
# y_test_pred = model.predict(X_test_scaled)

### Crear archivo de predicción

In [ ]:
# submission = pd.DataFrame({"laptop_ID": test_df["laptop_ID"], "Price_in_euros": y_test_pred})
# submission.to_csv("submission.csv", index=False)

In [ ]:
# print("Archivo submission.csv generado correctamente.")

Archivo submission.csv generado correctamente.
